### Notebook for preprocessing and analysis for Two-handed Task
- preprocess data
- run SVD
- Ouptout overview Graphs as png and Results as Csv

In [ ]:
import errno  # handy system and path functions
import glob
import locale
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import os

import pandas as pd
import sys  # to get file system encoding
import scipy.linalg
from scipy import signal
from scipy import stats
import seaborn as sns
from sklearn import preprocessing


# set Matplotlib parameters globally
params = {'legend.fontsize': 12,
          'legend.handlelength': 2,
         'figure.autolayout': True,
         'font.serif' : 'Ubuntu',
         'font.family' : 'serif',
         'font.monospace' : 'Ubuntu Mono',
         'font.size' : 12,
         'axes.labelsize' : 18,
         'axes.titlesize' : 20,
         'xtick.labelsize' : 20,
         'ytick.labelsize' : 20,
         'figure.titlesize' : 12,
         'figure.dpi' : 300}

rcParams.update(params)


# set Seaborn parameters globally
sns.set_context("paper")
sns.set_style('whitegrid')
sns.set_palette('colorblind')

In [ ]:
# script containing utility functions to preprocess and analyse data
%run ba_analysis_2h_functions.py

In [ ]:
marker_dict.values()

In [ ]:
def get_model(dimensions, event_file, hand, data_file):
    '''function to extract and resample movement events,
    calculate mean and variance of trahectories for each kinematic dimension
    and zero-center trajectories for further use in a SVD
    
    input: 
        dimensions = dimensionality of a model; One dimension consists of 1 kinematic dimension + one cartesian dimension
                    e.g. full scale model for right arm:
                    dimensions = right_hand_marker[:1] + right_wrist + right_lower_arm  + right_upper_arm + right_shoulder 
        event_file = dataframe containing information on which trials to include
        hand = string; either 'r' indicating right hand or 'l' indicating the left hand
        data_file = Dataframe containing all recorded movement data
        
    output:
        model containing zero-centered trajectories for specified dimensions
    '''

    df_model_a = pd.DataFrame()

    for i in dimensions:
        i = str(i)

        marker = ['marker_'+i+'_x', 'marker_'+i+'_y', 'marker_'+i+'_z']

        df_x, df_y, df_z = extract_and_resample_2H(event_file, data_file, marker, hand)
        x_mean_dimension, x_variance_dimension, x_df_corrected = get_corrected('x', df_x)
        y_mean_dimension, y_variance_dimension, y_df_corrected = get_corrected('y', df_y)
        z_mean_dimension, z_variance_dimension, z_df_corrected = get_corrected('z', df_z)

        reshaped_df = reshape(x_df_corrected, y_df_corrected, z_df_corrected)

        for idx, j in enumerate(reshaped_df.columns):
            df_model_a[str(marker[idx])] = reshaped_df[j]
    return df_model_a


In [ ]:
def specify_model(model, parameters):
    '''
    Function to further specify which dimensions of full scale model created by get_model function 
    to use for further analysis.
    
    input:
        model = model containing zero-centered trajectories for dimensions of interest
        parameters = list of ones and zeros indicating which kinnematich degree of freedom to extract
        
    output:
        list of parameters by which Dataframe containing full scale model can be truncated.
    '''
    model_parameters = []
    for idx, i in enumerate(parameters):  
        if i == 1:  # if parameter should be included it is encoded with a 1
            model_parameters.append(model.columns[idx])
    return model_parameters
    

In [ ]:
def plotting(marker_, pos, pos_name, subject_id, path, data_file):
    '''
    Output Trajectories, Variance and Velocity of Specified hand as csv and plot them.
    Perform SVD and plot mean loadings of first vector of U and Vt Matrix.
    
    Input:
        marker_ = Integer indicating which markers trajectory should be extracted
        pos= DataFrame containing all events for specific target location
        pos_name = identifier of target position
        subject_id = subject identifier
        data_file = Dataframe containing all recorded movement data of one subject
    
    '''
    # create subj directory
    directory = path+'/graphs/2H/'+subject_id+'/'+pos_name
    try:
        os.makedirs(directory+'/')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    # create directory for ouputs on  velocity
    try:
        os.makedirs(directory+'/velocity/')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    # create directory for ouputs on  trajectories
    try:
        os.makedirs(directory+'/trajectories/')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    # extract specifics of target postion
    box_pos_x = -1*(pos['box_r_pos_x'][0])
    box_pos_z_r = pos['box_r_pos_z'][0]
    box_pos_z_l = pos['box_l_pos_z'][0]
    box_pos_y_r = pos['box_r_pos_y'][0]
    box_pos_y_l = pos['box_l_pos_y'][0]

    f_size = 16  # font size for certain graphs
    tick_size = 15  # tick size for certain graphs
    
# RIGHT HAND DATA EXTRACTION
    i = marker_[0] #  right hand marker
    # define Marker as dictionary with entry for marker in each cartesian dimension
    marker_r =['marker_'+i+'_x', 'marker_'+i+'_y', 'marker_'+i+'_z']
    hand = 'r'
    # Ectract and resample movement date for trials with specififed target position for specified marker
    df_x_r, df_y_r, df_z_r = extract_and_resample_2H(pos, data_file, marker_r, hand)
    # extract mean trajectory, variance, and zero-centered trajectory for each cartesian dimension
    x_mean_dimension_r, x_variance_dimension_r, x_df_corrected_r = get_corrected('x', df_x_r)
    y_mean_dimension_r, y_variance_dimension_r, y_df_corrected_r = get_corrected('y', df_y_r)
    z_mean_dimension_r, z_variance_dimension_r, z_df_corrected_r = get_corrected('z', df_z_r)
    reshaped_df_r = reshape(x_df_corrected_r, y_df_corrected_r, z_df_corrected_r)

    # output extracted data as csvs
    df_trajectories_r = pd.DataFrame()
    df_trajectories_r['x_dim_'+subject_id+'_'+pos_name] = x_mean_dimension_r
    df_trajectories_r['y_dim_'+subject_id+'_'+pos_name] = y_mean_dimension_r
    df_trajectories_r['z_dim_'+subject_id+'_'+pos_name] = z_mean_dimension_r
    df_trajectories_r['x_dim_var_'+subject_id+'_'+pos_name] = x_variance_dimension_r
    df_trajectories_r['y_dim_var_'+subject_id+'_'+pos_name] = y_variance_dimension_r
    df_trajectories_r['z_dim_var_'+subject_id+'_'+pos_name] = z_variance_dimension_r
    df_trajectories_r['box_pos_x_r_'+subject_id+'_'+pos_name] = box_pos_x
    df_trajectories_r['box_pos_z_r_'+subject_id+'_'+pos_name] = box_pos_z_r
    df_trajectories_r['box_pos_y_r_'+subject_id+'_'+pos_name] = box_pos_y_r
    df_trajectories_r.to_csv(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_trajectories_right_hand.csv', index=False, encoding='utf-8')
    reshaped_df_r.to_csv(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_corrected_trajectories_right_hand.csv', index=False, encoding='utf-8')


# LEFT HAND DATA EXTRACTION
    i = marker_[1]  #  left hand marker
    marker_l =['marker_'+i+'_x', 'marker_'+i+'_y', 'marker_'+i+'_z']
    hand = 'l'
    # Ectract and resample movement date for trials with specififed target position for specified marker
    df_x_l, df_y_l, df_z_l = extract_and_resample_2H(pos, data_file, marker_l, hand)
    # extract mean trajectory, variance, and zero-centered trajectory for each cartesian dimension
    x_mean_dimension_l, x_variance_dimension_l, x_df_corrected_l = get_corrected('x', df_x_l)
    y_mean_dimension_l, y_variance_dimension_l, y_df_corrected_l = get_corrected('y', df_y_l)
    z_mean_dimension_l, z_variance_dimension_l, z_df_corrected_l = get_corrected('z', df_z_l)
    reshaped_df_l = reshape(x_df_corrected_l, y_df_corrected_l, z_df_corrected_l)

    # output extracted data as csvs
    df_trajectories_l = pd.DataFrame()
    df_trajectories_l['x_dim_'+subject_id+'_'+pos_name] = x_mean_dimension_l
    df_trajectories_l['y_dim_'+subject_id+'_'+pos_name] = y_mean_dimension_l
    df_trajectories_l['z_dim_'+subject_id+'_'+pos_name] = z_mean_dimension_l
    df_trajectories_l['x_dim_var_'+subject_id+'_'+pos_name] = x_variance_dimension_l
    df_trajectories_l['y_dim_var_'+subject_id+'_'+pos_name] = y_variance_dimension_l
    df_trajectories_l['z_dim_var_'+subject_id+'_'+pos_name] = z_variance_dimension_l
    df_trajectories_l['box_pos_x_l_'+subject_id+'_'+pos_name] = box_pos_x
    df_trajectories_l['box_pos_z_l_'+subject_id+'_'+pos_name] = box_pos_z_l
    df_trajectories_l['box_pos_y_l_'+subject_id+'_'+pos_name] = box_pos_y_l
    df_trajectories_l.to_csv(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_trajectories_left_hand.csv', index=False, encoding='utf-8')
    reshaped_df_l.to_csv(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_corrected_trajectories_left_hand.csv', index=False, encoding='utf-8')
    
    # Transform data into np-arrays for easier plotting
    # mean trajectory right hand
    x = np.array(x_mean_dimension_r)
    z=  np.array(z_mean_dimension_r)
    y = np.array(y_mean_dimension_r)

    # lower boundaries of variance right hand
    x_l = x - np.array(x_variance_dimension_r)
    z_l = z - np.array(z_variance_dimension_r)
    y_l  = y - np.array(y_variance_dimension_r)

    # upper boundaries of variance right hand
    x_u = x + np.array(x_variance_dimension_r)
    z_u = z + np.array(z_variance_dimension_r)
    y_u  = y + np.array(y_variance_dimension_r)

    # mean trajectory left hand
    l_x = np.array(x_mean_dimension_l)
    l_z=  np.array(z_mean_dimension_l)
    l_y = np.array(y_mean_dimension_l)
    
    # lower boundaries of variance left hand
    l_x_l = l_x - np.array(x_variance_dimension_l)
    l_z_l = l_z - np.array(z_variance_dimension_l)
    l_y_l  = l_y - np.array(y_variance_dimension_l)

    # upper boundaries of variance left hand
    l_x_u = l_x + np.array(x_variance_dimension_l)
    l_z_u = l_z + np.array(z_variance_dimension_l)
    l_y_u  = l_y + np.array(y_variance_dimension_l)
    ## Plot Velocity
    df_velocities = pd.DataFrame()

    ## Plot Velocity for specifed marker in each cartesian dimension
# RIGHT HAND X-Dimension
    fig = plt.figure(figsize=(12,6))
    velocity = np.diff(x_mean_dimension_r)/np.diff(range(len(x_mean_dimension_r)))
    df_velocities['x_dim_velocitiy_r_'+str(subject_id)+'_'+str(pos_name)] = velocity

    plt.plot(range(len(velocity)),velocity, color='crimson', label='Right Hand')
    plt.ylim(-0.05,0.05)
    plt.xlabel('Zeit/Samples', fontsize=f_size)
    plt.ylabel('Beschleunigung(m/sample)', fontsize=f_size)
# LEFT HAND X-Dimension
    velocity = np.diff(x_mean_dimension_l)/np.diff(range(len(x_mean_dimension_l)))
    df_velocities['x_dim_velocitiy_l_'+str(subject_id)+'_'+str(pos_name)] = velocity
    plt.plot(range(len(velocity)),velocity, color='royalblue', label='Linke Hand')
    plt.ylim(-0.05,0.05)
    plt.xlabel('Zeit/Samples',fontsize=f_size)
    plt.ylabel('Beschleunigung(m/sample)',fontsize=f_size)
    plt.title('Mittlere Beschleunigung in X-Dimension; Targetposition: '+ str(box_pos_x),fontsize=f_size)
    plt.legend()
    plt.tick_params(labelsize=tick_size)

    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/velocity/'+pos_name+'_velocity_x.png')
    plt.show()

# RIGHT HAND Y-Dimension
    fig = plt.figure(figsize=(12,6))
    velocity = np.diff(y_mean_dimension_r)/np.diff(range(len(y_mean_dimension_r)))
    df_velocities['y_dim_velocitiy_r_'+str(subject_id)+'_'+str(pos_name)] = velocity

    plt.plot(range(len(velocity)),velocity, color='crimson', label='Rechte Hand')
    plt.ylim(-0.05,0.05)
    plt.xlabel('Zeit/Samples', fontsize=f_size)
    plt.ylabel('Mittlere Beschleunigung(m/sample)', fontsize=f_size)
# LEFT HAND Y-Dimension 
    velocity = np.diff(y_mean_dimension_l)/np.diff(range(len(y_mean_dimension_l)))
    df_velocities['y_dim_velocitiy_l_'+str(subject_id)+'_'+str(pos_name)] = velocity

    plt.plot(range(len(velocity)),velocity, color='royalblue', label='Linke Hand')
    plt.ylim(-0.05,0.05)
    plt.xlabel('Zeit/Samples', fontsize=f_size)
    plt.ylabel('Beschleunigung(m/sample)', fontsize=f_size)
    plt.title('Mittlere Beschleunigung in Y-Dimension; Targetpositionen: '+ 'R: ['+str(box_pos_y_r) +
              '] ; L: ['+str(box_pos_y_l)+']', fontsize=f_size)
    plt.legend()
    plt.tick_params(labelsize=tick_size)
    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/velocity/'+pos_name+'_velocity_y.png')
    plt.show()

# RIGHT HAND Z-Dimension
    fig = plt.figure(figsize=(12,6))
    velocity = np.diff(z_mean_dimension_r)/np.diff(range(len(z_mean_dimension_r)))
    df_velocities['z_dim_velocitiy_r_'+str(subject_id)+'_'+str(pos_name)] = velocity

    plt.plot(range(len(velocity)),velocity, color='crimson', label='Rechte Hand')
    plt.ylim(-0.05,0.05)
    plt.xlabel('Zeit/Samples', fontsize=f_size)
    plt.ylabel('Mittlere Beschleunigung(m/sample)', fontsize=f_size)
# LEFT HAND Z-Dimension 
    velocity = np.diff(z_mean_dimension_l)/np.diff(range(len(z_mean_dimension_l)))
    df_velocities['z_dim_velocitiy_l_'+str(subject_id)+'_'+str(pos_name)] = velocity

    plt.plot(range(len(velocity)),velocity, color='royalblue', label='Linke Hand')
    plt.ylim(-0.05,0.05)
    plt.xlabel('Zeit/Samples', fontsize=f_size)
    plt.ylabel('Beschleunigung(m/sample)', fontsize=f_size)
    plt.title('Mittlere Beschleunigung in Z-Dimension; Targetpositionen: '+ 'R: ['+str(box_pos_z_r) +
              '] ; L: ['+str(box_pos_z_l)+']', fontsize=f_size)
    plt.legend()
    plt.tick_params(labelsize=tick_size)

    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/velocity/'+pos_name+'_velocity_z.png')
    plt.show()
    # save velocity data as csv
    df_velocities.to_csv(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/velocity/'+pos_name+'_velocities.csv', index=False, encoding='utf-8')


    ## PLOT TRAJECTORIES 2D    

    ## RIGHT HAND X-Dimension
    # Plot right hand trajextory
    fig = plt.figure(figsize=(14,7))
    plt.plot( range(0,len(x_mean_dimension_r)), 
                            x_mean_dimension_r, color='crimson', label='Rechte Hand')
    # PLOT Single trials as shadows
    for i in df_x_r.columns:
        plt.plot(range(len(df_x_r[i])),df_x_r[i], alpha=0.1, color='red')

    # PLOT VARIANCE
    plt.plot(range(0,len(x_variance_dimension_r)),
                      x_l, color='black', alpha=0.75)
    plt.plot(range(0,len(x_variance_dimension_r)),
             (x_u), color='black', alpha=0.75)
    # FILL IN VARIANCE
    plt.fill_between(range(len(x_mean_dimension_r)), x_mean_dimension_r,(x_l),color='grey',alpha=.2)
    plt.fill_between(range(len(x_mean_dimension_r)), x_mean_dimension_r,x_u, color='grey',alpha=.2)

    ## LEFT HAND X-Dimension
    # Plot left hand trajextory
    plt.plot( range(0,len(x_mean_dimension_l)), 
                            x_mean_dimension_l, color='royalblue', label='left hand')
    # PLOT Single trials as shadows
    for i in df_x_l.columns:
        plt.plot(range(len(df_x_l[i])),df_x_l[i], alpha=0.1, color='blue')

    # PLOT VARIANCE
    plt.plot(range(0,len(x_variance_dimension_r)),
                      l_x_l, color='black', alpha=0.75)
    plt.plot(range(0,len(x_variance_dimension_r)), (l_x_u), color='black', alpha=0.75)

    # FILL IN VARIANCE
    plt.fill_between(range(len(x_mean_dimension_r)), x_mean_dimension_l,(l_x_l),color='grey',alpha=.2)
    plt.fill_between(range(len(x_mean_dimension_r)), x_mean_dimension_l,l_x_u, color='grey',alpha=.2)

    # define legend and title
    blue_line = mlines.Line2D([], [], color='royalblue', label='Linke Hand')
    red_line = mlines.Line2D([], [], color='crimson', label='Rechte Hand')
    plt.legend(handles=[blue_line, red_line])
    plt.title('Mittlere Trajektorie in X-Dimension; Targetposition: '+str(box_pos_x),fontsize=f_size)
    plt.xlabel('Samples', fontsize=f_size)
    plt.ylabel('X-Koordinaten(Meter)', fontsize=f_size)

    # plot target box
    plt.plot((0,0), (box_pos_x-.1, box_pos_x + .1), color='k',marker='x')
    plt.plot((0,len(x_mean_dimension_r)), (box_pos_x + .1, box_pos_x +.1), color='k',marker='x')
    plt.plot((0,len(x_mean_dimension_r)), (box_pos_x-.1, box_pos_x-.1), color='k',marker='x')
    plt.plot((len(x_mean_dimension_r),len(x_mean_dimension_r)), (box_pos_x-.1, box_pos_x + .1), color='k',marker='x')
    plt.tick_params(labelsize=tick_size)
    plt.ylim(-.1,1.1)

    # save and show figure
    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_x_coordinates.png')   
    plt.show()

    # Y-DIM
    fig = plt.figure(figsize=(14,7))
    # PLOT R TRAJCETORY
    plt.plot( range(0,len(y_mean_dimension_r)), 
                            y_mean_dimension_r, color='crimson')
    # PLOT TRIAL SHADOWS
    for i in df_y_r.columns:
        plt.plot(range(len(df_y_r[i])),df_y_r[i], alpha=0.1, color='red')


    ## PLOT VARIANCE
    plt.plot(range(0,len(x_variance_dimension_r)),
                      y_l, color='black', alpha=0.75)
    plt.plot(range(0,len(x_variance_dimension_r)),
             (y_u), color='black', alpha=0.75)
    # FILL IN VARIANCE
    plt.fill_between(range(len(x_mean_dimension_l)), y_mean_dimension_r,(y_l),color='grey',alpha=.2)
    plt.fill_between(range(len(y_mean_dimension_l)), y_mean_dimension_r,y_u, color='grey',alpha=.2)

    # PLOT TRIAL SHADOWS
    for i in df_y_l.columns:
        plt.plot(range(len(df_y_l[i])),df_y_l[i], alpha=0.1, color='blue')

    plt.plot( range(0,len(y_mean_dimension_l)), 
                            y_mean_dimension_l, color='royalblue')

    plt.plot(range(0,len(y_variance_dimension_l)),
                      l_y_l, color='black', alpha=0.75)
    plt.plot(range(0,len(y_variance_dimension_l)),
             (l_y_u), color='black', alpha=0.75)
    # FILL IN VARIANCE
    plt.fill_between(range(len(x_mean_dimension_l)), y_mean_dimension_l,(l_y_l),color='grey',alpha=.2)
    plt.fill_between(range(len(y_mean_dimension_l)), y_mean_dimension_l,l_y_u, color='grey',alpha=.2)


    #  get dummy variable for color in legend
    blue_line = mlines.Line2D([], [], color='royalblue', label='Linke Hand')
    red_line = mlines.Line2D([], [], color='crimson', label='Rechte Hand')

    #  plot legend
    plt.legend(handles=[blue_line, red_line])
    plt.ylabel('Y-Dimension(Meter)', fontsize=f_size)
    plt.xlabel('Zeit/Samples', fontsize=f_size)
    plt.title('Y-Koordinaten; Targetpositionen: '+ 'R: ['+str(box_pos_y_r) +
              '] ; L: ['+str(box_pos_y_l)+']', fontsize=f_size)


    # plot target box r
    plt.plot((0,0), (box_pos_y_r-.1, box_pos_y_r + .1), color='k',marker='x')
    plt.plot((0,len(y_mean_dimension_l)), (box_pos_y_r + .1, box_pos_y_r +.1), color='k',marker='x')
    plt.plot((0,len(y_mean_dimension_l)), (box_pos_y_r-.1, box_pos_y_r-.1), color='k',marker='x')
    plt.plot((len(y_mean_dimension_l),len(y_mean_dimension_l)), (box_pos_y_r-.1, box_pos_y_r + .1), color='k',marker='x')

    # plot target box l
    plt.plot((0,0), (box_pos_y_l-.1, box_pos_y_l + .1), color='k',marker='x')
    plt.plot((0,len(y_mean_dimension_l)), (box_pos_y_l + .1, box_pos_y_l +.1), color='k',marker='x')
    plt.plot((0,len(y_mean_dimension_l)), (box_pos_y_l-.1, box_pos_y_l-.1), color='k',marker='x')
    plt.plot((len(y_mean_dimension_l),len(y_mean_dimension_l)), (box_pos_y_l-.1, box_pos_y_l + .1), color='k',marker='x')
    plt.tick_params(labelsize=tick_size)
    plt.ylim(0.8,2)

    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_y_coordinates.png')

    plt.show()

    # Z-DIM
    fig = plt.figure(figsize=(14,7))
    # PLOT R TRAJCETORY
    plt.plot( range(0,len(z_mean_dimension_r)), 
                            z_mean_dimension_r, color='crimson')

    # PLOT TRIAL SHADOWS
    for i in df_z_r.columns:
        plt.plot(range(len(df_z_r[i])),df_z_r[i], alpha=0.1, color='red')

    ## PLOT VARIANCE
    plt.plot(range(0,len(x_variance_dimension_r)),
                      z_l, color='black', alpha=0.4)
    plt.plot(range(0,len(x_variance_dimension_r)), (z_u), color='black', alpha=0.4)

    # FILL IN VARIANCE
    plt.fill_between(range(len(x_mean_dimension_l)), z_mean_dimension_r,(z_l),color='grey',alpha=.2)
    plt.fill_between(range(len(y_mean_dimension_l)), z_mean_dimension_r,z_u, color='grey',alpha=.2)

    # PLOT L TRAJCETORY
    plt.plot( range(0,len(z_mean_dimension_l)), 
                            z_mean_dimension_l, color='royalblue')

    # PLOT TRIAL SHADOWS
    for i in df_z_l.columns:
        plt.plot(range(len(df_z_l[i])),df_z_l[i], alpha=0.1, color='blue')



    ## PLOT VARIANCE
    plt.plot(range(0,len(z_variance_dimension_l)),
                      l_z_l, color='black', alpha=0.75)
    plt.plot(range(0,len(z_variance_dimension_l)),
                         (l_z_u), color='black', alpha=0.75)

    # FILL IN VARIANCE
    plt.fill_between(range(len(x_mean_dimension_l)), z_mean_dimension_l,(l_z_l),color='grey',alpha=.2)
    plt.fill_between(range(len(y_mean_dimension_l)), z_mean_dimension_l,l_z_u, color='grey',alpha=.2)

    blue_line = mlines.Line2D([], [], color='royalblue', label='Linke Hand')
    red_line = mlines.Line2D([], [], color='crimson', label='Rechte Hand')

    plt.legend(handles=[blue_line, red_line])
    plt.ylabel('Z-Dimension(Meter)', fontsize=f_size)
    plt.xlabel('Zeit/Samples', fontsize=f_size)
    plt.title('Z-Koordinaten; Targetpositionen: '+ 'R: ['+str(box_pos_z_r) +
              '] ; L: ['+str(box_pos_z_l)+']', fontsize=f_size)

    # plot target box r
    plt.plot((0,0), (box_pos_z_r-.1, box_pos_z_r + .1), color='k',marker='x')
    plt.plot((0,len(z_mean_dimension_r)), (box_pos_z_r + .1, box_pos_z_r +.1), color='k',marker='x')
    plt.plot((0,len(z_mean_dimension_r)), (box_pos_z_r-.1, box_pos_z_r-.1), color='k',marker='x')
    plt.plot((len(z_mean_dimension_r),len(z_mean_dimension_r)), (box_pos_z_r-.1, box_pos_z_r + .1), color='k',marker='x')

    # plot target box l
    plt.plot((0,0), (box_pos_z_l-.1, box_pos_z_l + .1), color='k',marker='x')
    plt.plot((0,len(z_mean_dimension_l)), (box_pos_z_l + .1, box_pos_z_l +.1), color='k',marker='x')
    plt.plot((0,len(z_mean_dimension_l)), (box_pos_z_l-.1, box_pos_z_l-.1), color='k',marker='x')
    plt.plot((len(z_mean_dimension_l),len(z_mean_dimension_l)), (box_pos_z_l-.1, box_pos_z_l + .1), color='k',marker='x')
    plt.tick_params(labelsize=tick_size)
 
    plt.ylim(-.6,.6)

    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_z_coordinates.png')

    plt.show()
    
    
    ### PLOT TRAJECTTORIES IN 3D
    fig = plt.figure(figsize=(16,9))
    # right hand
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(xs=x, ys=z, zs=y, color='crimson', label='Rechte Hand')
    trajectory_r_x = x
    trajectory_r_z = z
    trajectory_r_y = y
    ax.plot(xs=x_u, ys=z_u, zs=y_u, color='black', alpha=0.6)
    ax.plot(xs=x_l, ys=z_l, zs=y_l, color='black', alpha=0.6)

    for idx, i in enumerate(df_x_r.columns):
        x_single = np.array(df_x_r[df_x_r.columns[idx]])
        z_single=  np.array(df_z_r[df_z_r.columns[idx]])
        y_single = np.array(df_y_r[df_y_r.columns[idx]])
        ax.plot(xs=x_single, ys=z_single, zs=y_single, color='grey', alpha=0.1)


    # left hand
    ax.plot(xs=l_x, ys=l_z, zs=l_y, color='royalblue', label='Linke Hand')
    trajectory_l_x = l_x
    trajectory_l_z = l_z
    trajectory_l_y = l_y
    ax.plot(xs=l_x_u, ys=l_z_u, zs=l_y_u, color='black', alpha=0.6)
    ax.plot(xs=l_x_l, ys=l_z_l, zs=l_y_l, color='black', alpha=0.6)

    for idx, i in enumerate(df_x_l.columns):
        x_single = np.array(df_x_l[df_x_l.columns[idx]])
        z_single=  np.array(df_z_l[df_z_l.columns[idx]])
        y_single = np.array(df_y_l[df_y_l.columns[idx]])
        ax.plot(xs=x_single, ys=z_single, zs=y_single, color='grey', alpha=0.1)


    ax.set_ylim(.5,-.6)
    ax.set_xlim(-0.2,1)
    ax.set_zlim(0.8,2)
    ax.view_init(azim=235)

    ax.set_xlabel('x', fontsize=23)
    ax.set_ylabel('z', fontsize=23)
    ax.set_zlabel('y', fontsize=23)
    plt.legend()
    plt.tick_params(labelsize=tick_size)

    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_3D_coordinates.png')

    plt.show()
    

    ### PLOT MEAN FACTOR LOADING OF u1

    n_elements = 15  # how many values should be extracted
    # define marker space
    dimensions_l = left_hand_marker[:1] + left_wrist + left_lower_arm  + left_upper_arm + left_shoulder 
    model_l = get_model(dimensions_l, pos, 'l', data_file)
    # specify which parameters to extract
    parameters = [1,1,1,0,0,0,0,0,0,0,0,0,0,0,0]
    model_parameters = specify_model(model_l, parameters)
    model_l = model_l[model_parameters]
    # SVD
    U_l, s_l, S_l, VT_l, mean_loadings_l, stdm_l, concatenated_l, var_explained_l = extract_mean_factor_loading(model_l)

    len_x = []
    for i in range(1,len(s_l)+1):
        len_x.append(i)

    fig1 = plt.figure(figsize=(10,5))
    sns.barplot(y=s_l, x=len_x)
    plt.title('Singulärwerte: Linke Hand', fontsize=f_size)
    plt.tick_params(labelsize=tick_size)

    plt.show()

    # show explained variance
    legend_elements = []
    for idx, i in enumerate(var_explained_l):
        element = Line2D([0], [0], alpha=0, lw=2, marker='o', markersize=15, label=('SV '+str(idx)+': '+str(var_explained_l[idx])))
        legend_elements.append(element)
    fig1 = plt.figure(figsize=(12,6))
    sns.barplot(x=list(range(1,len(var_explained_l)+1)),
            y=var_explained_l)
    plt.xlabel('Singulärwerte: Linke Hand', fontsize=16)
    plt.ylabel('Aufgeklärte Varianz(%)', fontsize=16)
    plt.ylim(0,1)
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 10})
    plt.tick_params(labelsize=tick_size)

    fig1.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_variance_explained_l.png')
    plt.show()


    n_elements = 15  # how many values should be extracted
    # define marker space
    dimensions_r = right_hand_marker[:1] + right_wrist + right_lower_arm  + right_upper_arm + right_shoulder 
    #dimensions = left_hand_marker[:1] + left_wrist + left_lower_arm  + left_upper_arm + left_shoulder 
    model_r = get_model(dimensions_r, pos, 'r', data_file)
    # specify which parameters to extract
    parameters = [1,1,1,0,0,0,0,0,0,0,0,0,0,0,0]
    model_parameters = specify_model(model_r, parameters)
    model_r = model_r[model_parameters]
    # SVD
    U_r, s_r, S_r, VT_r, mean_loadings_r, stdm_r, concatenated_r, var_explained_r = extract_mean_factor_loading(model_r)

    len_x = []
    for i in range(1,len(s_r)+1):
        len_x.append(i)

    fig1 = plt.figure(figsize=(10,5))
    sns.barplot(y=s_r, x=len_x)
    plt.title('Singulärwerte: Rechte Hand', fontsize=f_size)
    plt.show()

    # show explained variance
    legend_elements = []
    for idx, i in enumerate(var_explained_r):
        element = Line2D([0], [0], alpha=0, lw=2, marker='o', markersize=15, label=('SV '+str(idx)+': '+str(var_explained_r[idx])))
        legend_elements.append(element)
    fig1 = plt.figure(figsize=(10,5))
    sns.barplot(x=list(range(1,len(var_explained_r)+1)),
            y=var_explained_r)
    plt.xlabel('Singulärwerte: Rechte Hand', fontsize=f_size)
    plt.ylabel('Aufgeklärte Varianz(%)',fontsize=f_size)
    plt.ylim(0,1)
    plt.tick_params(labelsize=tick_size)
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 10})
    fig1.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_variance_explained_r.png')

    plt.show()

    #plot mean loadings
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 8))
    ## RIGHT HAND u1
    ax.plot(range(len(mean_loadings_r)), mean_loadings_r,color='crimson', label ='Rechte Hand')
    ax.plot(range(len(stdm_r)), mean_loadings_r+stdm_r, color='black', alpha=0.6)
    ax.plot(range(len(stdm_r)), mean_loadings_r-stdm_r, color='black', alpha=0.6)
    ax.fill_between(range(len(mean_loadings_r)), mean_loadings_r,(mean_loadings_r+stdm_r),color='red',alpha=.2)
    ax.fill_between(range(len(mean_loadings_r)), mean_loadings_r,mean_loadings_r-stdm_r, color='red',alpha=.2)

    ax.set_xlabel('Zeit/Samples', fontsize=f_size)
    ax.set_ylabel('Mittlere Faktorenladung', fontsize=f_size)

    ## LEFT HAND u1
    ax.plot(range(len(mean_loadings_l)), mean_loadings_l,color='royalblue', label ='left hand')
    ax.plot(range(len(stdm_l)), mean_loadings_l+stdm_l, color='black', alpha=0.6)
    ax.plot(range(len(stdm_l)), mean_loadings_l-stdm_l, color='black', alpha=0.6)
    ax.fill_between(range(len(mean_loadings_l)), mean_loadings_l,(mean_loadings_l+stdm_l),color='royalblue',alpha=.2)
    ax.fill_between(range(len(mean_loadings_l)), mean_loadings_l,mean_loadings_l-stdm_l, color='royalblue',alpha=.2)

    ax.set_xlabel('Zeit/Samples',fontsize=f_size)
    ax.set_ylabel('Mittlere Faktorenladung')
    plt.ylim(0,.05)
    plt.legend()
    plt.title('Mittlere Faktorenladung auf Singulärvektor u1', fontsize=f_size)
    plt.tick_params(labelsize=tick_size)
    
    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_u1_mean_loadings.png')


    ## PLOT FIRST SINGULAR VECTOR OF V
    sv_r = np.abs(VT_r[0,:])
    sv_l = np.abs(VT_l[0,:])

    fig = plt.figure(figsize=(14,7))
    # right hand

    ax = fig.add_subplot(111, projection='3d')
    ax.plot(xs=(0,sv_l[0]), ys=(0,sv_l[2]), zs=(0,sv_l[1]), color='royalblue',
           label = ('Linke Hand: v1; [x]='+str(sv_l[0])+' ; [y]='+str(sv_l[1])+' ; [z]='+str(sv_l[2])))
    ax.quiver(0,0, 0,sv_l[0],sv_l[2], sv_l[1], length = 1, arrow_length_ratio=0.1,normalize = True, color='royalblue')

    ax.plot(xs=(0,sv_r[0]), ys=(0,sv_r[2]), zs=(0,sv_r[1]), color='crimson',
           label = ('Rechte Hand: v1; [x]='+str(sv_r[0])+' ; [y]='+str(sv_r[1])+' ; [z]='+str(sv_r[2])))
    ax.quiver(0,0, 0,sv_r[0],sv_r[2], sv_r[1], length = 1, arrow_length_ratio=0.1,normalize = True, color='crimson')

    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_zlim(0,1)

    ax.set_xlabel('x', fontsize=23)
    ax.set_ylabel('z', fontsize=23)
    ax.set_zlabel('y', fontsize=23)
    ax.view_init(azim=230)

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
              fancybox=True, shadow=True)
    plt.tick_params(labelsize=tick_size)
    plt.title('Ausrichtung des Singulärvektos v1', y=-0.03, fontsize=f_size)
    fig.savefig(path+'/graphs/2H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_v1.png')

    return trajectory_r_x,trajectory_r_y, trajectory_r_z , trajectory_l_x, trajectory_l_y, trajectory_l_z, mean_loadings_r,  mean_loadings_l
 

In [ ]:
dir_ = '/home/michael/bachelorarbeit/subjects/*'  # input directory; 1H/ -> Sub-01, Sub-02, .. 
path = '/home/michael/bachelorarbeit'  # output directory 

# create lists for data on each hand and dimension (x,y,z)
list_trajectories_r_x = []
list_trajectories_r_y = []
list_trajectories_r_z = []
list_trajectories_l_x = []
list_trajectories_l_y = []
list_trajectories_l_z = []
list_mean_loadings_r = []
list_mean_loadings_l = []

subjects = []  # list to keep track of subjects 
subjects_info = pd.DataFrame() # Setup Dataframe containing handedness and height of subject
for subject in (glob.glob(dir_)): # iterate through subject level folders
    subject_id = subject.split('/')[5]  # subject identifier
    print(subject_id)

    # exclude bad subjects
    if subject_id == 'sub-03':
        print('excluded Subject')
    elif subject_id == 'sub-02':
        print('excluded Subject')
    elif subject_id == 'sub-10':
        for filename in (glob.glob(subject+'/*')):
            if filename.split('_')[2] == 'parameters':
                print('info')
                subject_parameters = pd.read_csv(filename)
                subjects_info = subjects_info.append(subject_parameters)


            elif filename.split('-')[3] == '2H' and filename.split('-')[4] == 'events':
                print('events')
                print(filename)
                event_file_2H = pd.read_csv(filename)
            elif filename.split('-')[3] == '2H' and filename.split('-')[4] == 'data':
                print('data')
                print(filename)
                data_file_2H = pd.read_csv(filename)
        r1_m1,r1_l1,m1_l1,r2_l2,m2_l2,r2_m2,r3_l3,m3_l3,r3_m3,r1_l3,r3_l1,m1_m3,m3_m1 = group_by_pos_both_hands(event_file_2H)
        positions_list = [r1_m1,r1_l1,m1_l1,r2_l2,m2_l2,r2_m2,r3_l3,m3_l3,r3_m3,r1_l3,r3_l1,m1_m3,m3_m1]
        pos_names = ['r1_m1','r1_l1','m1_l1','r2_l2','m2_l2','r2_m2','r3_l3','m3_l3','r3_m3','r1_l3','r3_l1','m1_m3','m3_m1']

        for idx, i in enumerate(positions_list):
        #     print(i)
            trajectory_r_x,trajectory_r_y, trajectory_r_z , trajectory_l_x, trajectory_l_y, trajectory_l_z, mean_loadings_r, mean_loadings_l = plotting(['7', '4'], i, pos_names[idx], subject_id, path, data_file_2H)
            list_trajectories_r_x.append(trajectory_r_x)
            list_trajectories_r_y.append(trajectory_r_y)
            list_trajectories_r_z.append(trajectory_r_z)
            list_trajectories_l_x.append(trajectory_l_x)
            list_trajectories_l_y.append(trajectory_l_y)
            list_trajectories_l_z.append(trajectory_l_z)
            list_mean_loadings_r.append(mean_loadings_r)
            list_mean_loadings_l.append(mean_loadings_l)


In [ ]:
## PLOT MEAN LOADINGS OF EACH HAND

df_mean_loadings = pd.DataFrame()
for i in range(len(list_mean_loadings_l)):
    df_mean_loadings['right_hand_' +str(i)] = list_mean_loadings_r[i]
    df_mean_loadings['left_hand_' +str(i)] = list_mean_loadings_l[i]
df_mean_loadings.to_csv('/home/michael/Desktop/xxxx.csv')
    
df_mean_loadings = df_mean_loadings.values
df_mean_loadings = np.absolute(df_mean_loadings)


mean_loadings = []
stdm = []
for i in range(len(df_mean_loadings)):
    mean_loadings.append(df_mean_loadings[i].mean())
    stdm.append(scipy.stats.sem(df_mean_loadings[i]))
    
mean_loadings = np.array(mean_loadings)
stdm = np.array(stdm)


fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 8))
f_size = 16
tick_size = 12

for i in range(len(list_mean_loadings_r)):
    ## RIGHT HAND u1
    ax.plot(range(len(list_mean_loadings_r[i])), list_mean_loadings_r[i],color='maroon', alpha=0.2)
    ax.set_xlabel('Samples', fontsize=f_size)
    ax.set_ylabel('Mittlere Faktorenladung', fontsize=f_size)

    ## LEFT HAND u1
    ax.plot(range(len(list_mean_loadings_l[i])), list_mean_loadings_l[i],color='navy', alpha=0.2)
    ax.set_xlabel('Samples',fontsize=f_size)
    ax.set_ylabel('Mittlere Faktorenladung')
    plt.ylim(0,.05)
    plt.legend()
    plt.title('Mittlere Faktorenladung auf Singulärvektor u1', fontsize=f_size)
    plt.tick_params(labelsize=tick_size)
    
## RIGHT HAND u1
ax.plot(range(len(mean_loadings)), mean_loadings,color='black', label ='Rechte Hand')
#ax.plot(range(len(stdm)), stdm, color='steelblue')
ax.plot(range(len(stdm)), mean_loadings+stdm, color='black', alpha=0.4)
ax.plot(range(len(stdm)), mean_loadings-stdm, color='black', alpha=0.4)
ax.fill_between(range(len(mean_loadings)), mean_loadings,(mean_loadings+stdm),color='grey',alpha=.2)
ax.fill_between(range(len(mean_loadings)), mean_loadings,mean_loadings-stdm, color='grey',alpha=.2)


# ax.set_title('subject '+str(subject_id)+ ' '+str(shelf_id))
ax.set_xlabel('Samples', fontsize=f_size)
ax.set_ylabel('Mittlere Faktorenladung', fontsize=f_size)
blue_line = mlines.Line2D([], [], color='royalblue', label='Linke Hand')
red_line = mlines.Line2D([], [], color='crimson', label='Rechte Hand')
black_line = mlines.Line2D([], [], color='Black', label='Mittlerwert der Faktoren')

trajectories_df = pd.DataFrame()
trajectories_df['left'] = list_mean_loadings_l
trajectories_df['right']= list_mean_loadings_r
trajectories_df.to_csv(path+'/graphs/'+subject_id+'_all_laodings.csv')

fig.savefig(path+'/graphs/'+subject_id+'_overall_u1_mean_loadings.png')
